# Testing DP0.3 ephemerides and coordinates
By Jamie Robinson

This notebook retrieves the observations and orbital parameters of an object in DP0.3. We compare the ephemerides and coordinates from DP0.3 to the output from propagating the orbit.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from astropy import units as u
from astropy.coordinates import SkyCoord, GCRS
from sbpy.data import Orbit
from astropy.time import Time
from astropy.coordinates import solar_system_ephemeris
from sbpy.photometry import HG
from astropy.table import QTable

from lsst.rsp import get_tap_service

In [ ]:
service = get_tap_service("ssotap")
assert service is not None

In [ ]:
# ssObjectId of object to analyse
ssoid = "6098332225018"  # good test object

In [ ]:
query = """
SELECT
    *
FROM
    dp03_catalogs_10yr.DiaSource as dia
INNER JOIN
    dp03_catalogs_10yr.SSSource as sss
ON
    dia.diaSourceId = sss.diaSourceId
WHERE
    dia.ssObjectId={}
ORDER by dia.ssObjectId
""".format(
    ssoid
)

df = service.search(query).to_table().to_pandas()
df

In [ ]:
results = service.search("SELECT * FROM dp03_catalogs_10yr.MPCORB " "WHERE ssObjectId={}".format(ssoid))
df_orb = results.to_table().to_pandas()

In [ ]:
df_orb

In [ ]:
# put obs in time order
df = df.sort_values("midPointMjdTai")

### Resample the sparse observations by propagating orbit with sbpy & oorb

In [ ]:
# check orbit
e = df_orb["e"]
incl = df_orb["incl"]
q = df_orb["q"]
a = q / (1.0 - e)
Q = a * (1.0 + e)
print(a, e, incl)

In [ ]:
df_orb["a"] = a
df_orb["Q"] = Q
df_orb

In [ ]:
# Calculate some extra orbital elements
df_orb["P"] = df_orb["a"] ** (3.0 / 2.0)  # orbital period in years
df_orb["n"] = 360.0 / (df_orb["P"] * 365.25)  # mean motion in deg/day
df_orb["M"] = (
    df_orb["n"] * (df_orb["epoch"] - df_orb["tperi"])
) % 360  # angles must be in correct range otherwise sbpy/pyoorb freak out
df_orb[["a", "P", "n", "M"]]

In [ ]:
# rename columns for consistency
df_orb = df_orb.rename(columns={"node": "Omega", "peri": "w"})

In [ ]:
df_orb[["a", "e", "incl", "Omega", "w", "M"]]

In [ ]:
# create an sbpy oorb object from dataframe via QTable
tab = QTable.from_pandas(
    df_orb[["a", "e", "incl", "Omega", "w", "M"]],
    units={"a": u.au, "incl": u.deg, "Omega": u.deg, "w": u.deg, "M": u.deg},
)
orbit = Orbit.from_table(tab)

In [ ]:
# oorb requires certain extra fields
orbit["epoch"] = Time(Time(df_orb["epoch"], format="mjd").jd, format="jd")
orbit["targetname"] = np.array(df_orb["ssObjectId"]).astype(str)
orbit["H"] = df_orb["mpcH"] * u.mag
orbit["G"] = df_orb["mpcG"] * u.dimensionless_unscaled

In [ ]:
orbit

In [ ]:
# define a set of JD times to propagate the orbital elements to
N = 1000
times = Time(
    Time(np.linspace(np.amin(df["midPointMjdTai"]), np.amax(df["midPointMjdTai"]), N), format="mjd").jd,
    format="jd",
)
times[0]

In [ ]:
# create an empty dataframe to hold resampled observations
df_dense = pd.DataFrame()
df_dense["midPointMjdTai"] = times.mjd

In [ ]:
# propagate the orbit forward in time.
# probably a better way to do this but I can't get oo_propagate to work with a Time list right now
# see: https://github.com/NASA-Planetary-Science/sbpy/issues/341

df_pos = pd.DataFrame()  # empty dataframe to hold cartesian coordinates

for i in range(len(times)):
    print(i)
    prop_elem = orbit.oo_propagate(times[i])  # propagate the orbit to the selected time step
    del prop_elem.table[
        "orbtype"
    ]  # orbtype is added as int, sbpy freaks out so delete the orbtype and then _to_oo works it out
    print("propagate")
    statevec = prop_elem.oo_transform("CART")  # transform from orbital elements to cartesian
    print("transform")

    # append new cartesian coordinates to the dataframe
    _df_statevec = statevec.table.to_pandas()
    df_pos = pd.concat((df_pos, _df_statevec))

df_pos.reset_index(drop=True, inplace=True)

In [ ]:
df_pos

### Use astropy coordinates to transform between all the coordinate systems

In [ ]:
# define heliocentric cartesian coordinates
c_xyz_hel = SkyCoord(
    x=np.array(df_pos["x"]),
    y=np.array(df_pos["y"]),
    z=np.array(df_pos["z"]),
    unit="AU",
    representation_type="cartesian",
    frame="heliocentrictrueecliptic",
)

In [ ]:
# transform to heliocentric ecliptic coords
c_ecl_hel = c_xyz_hel.copy()
c_ecl_hel.representation_type = "spherical"

In [ ]:
# transform to geocentric equatorial coords (times required to calculate Earth position)
with solar_system_ephemeris.set("jpl"):
    c_eq_geo = c_xyz_hel.transform_to(GCRS(obstime=times))

In [ ]:
# transform to geocentric cartesian coords
c_xyz_geo = c_eq_geo.copy()
c_xyz_geo.representation_type = "cartesian"

In [ ]:
# transform from geo equatorial (ra, dec) to geo ecliptic (lon, lat)
c_ecl_geo = c_eq_geo.transform_to("geocentrictrueecliptic")

In [ ]:
# plot the propagated cartesian positions against the database values

x_plot = "midPointMjdTai"
y_plot1 = "heliocentricX"
y_plot2 = "heliocentricY"
y_plot3 = "heliocentricZ"
df_plot = df

fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0, 0])

x = ax1.scatter(df_plot[x_plot], df_plot[y_plot1], label=y_plot1)
x = ax1.scatter(df_plot[x_plot], df_plot[y_plot2], label=y_plot2)
x = ax1.scatter(df_plot[x_plot], df_plot[y_plot3], label=y_plot3)

ax1.plot(times.mjd, c_xyz_hel.x)
ax1.plot(times.mjd, c_xyz_hel.y)
ax1.plot(times.mjd, c_xyz_hel.z)

ax1.set_xlabel(x_plot)
ax1.set_ylabel("distance")
ax1.legend()

plt.show()

There are some deviations of x and y positions, probably due to slightly different reference frames and methods of propagating orbits.

There is something wrong with database z positions, will be fixed soon!

In [ ]:
# the ecliptic coordinates look good!

x_plot = "midPointMjdTai"
y_plot1 = "eclipticLambda"
y_plot2 = "eclipticBeta"
df_plot = df

fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0, 0])

x = ax1.scatter(df_plot[x_plot], df_plot[y_plot1], label=y_plot1)
x = ax1.scatter(df_plot[x_plot], df_plot[y_plot2], label=y_plot2)

ax1.plot(times.mjd, c_ecl_geo.lon.degree)
ax1.plot(times.mjd, c_ecl_geo.lat.degree)

ax1.set_xlabel(x_plot)
ax1.set_ylabel("angle")
ax1.legend()

plt.show()